In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.cuda.current_device())
import pandas as pd
import string
from utils.preprocess import DataPreProcessor
import gensim.downloader as api
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import random

# Load data
eng_emails_raw = pd.read_pickle('Data/en_emails_raw.pkl')
# ch_emails_raw = pd.read_pickle('Data/ch_emails_raw.pkl')

True
NVIDIA GeForce RTX 2060
0


In [3]:
eng_emails_raw.head()

,class,contains_img,email_from,email_to,message,subject,subpath
1,spam,False,"""Stella Lowry"" <rookcuduq@yahoo.com>","""Brian"" <bernice@groucho.cs.psu.edu>",\n ...,re[12]:,../data/000/001
2,spam,False,"""Walter"" <trwmpca@downtowncumberland.com>",<arline@groucho.cs.psu.edu>,Academic Qualifications available from prestig...,Take a moment to explore this.,../data/000/002
3,ham,False,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,Greetings all. This is to verify your subscri...,Greetings,../data/000/003
4,spam,False,"""Mr Jailyn Koepke"" <kiflsbizc@attheworld.com>",melvin@groucho.cs.psu.edu,try chauncey may conferred the luscious not co...,LOANS @ 3.17% (27 term),../data/000/004
5,ham,False,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,"It's quiet. Too quiet. Well, how about a str...",who wants to start?,../data/000/005


In [ ]:
class ModelPreprocessor(DataPreProcessor):
    def __init__(self):
        self.word2vec_corpus = api.load('word2vec-google-news-300')
        super().__init__() # base class doesn't tokenize

    def tokenize(self,doc):
        """
        function to tokenize text into constituent words, using NLTK API.
        return list of tokens from input text: List[str]
        """
        text = doc["pipe_text"]
        tokens = word_tokenize(text)
        return tokens

    #function to perform search against Word2Vec
    def get_word2vec_embedding(self,token):
        try:
            token_embedding = self.word2vec_corpus[token]
            return token_embedding
        except:
            return False
    
    #function to perform search against WordNet
    def wordnet_replacement(self,token):
        """
        function to generate a synonym of input token using Wordnet
        input: token
        conditions: token must not have a Word2Vec embedding
        output: replacement token -> str
        """
        # token should not be found in word2vec
        assert self.get_word2vec_embedding(token) == False
        synonyms = []
        synsets = wn.synsets(token)
        if len(synsets) == 0: return []
        for s in synsets:
            lemma_names = s.lemma_names()
            for ln in lemma_names:
                ln = ln.lower().replace('_',' ')
                if (ln != token) and (ln not in synonyms):
                    synonyms.append(ln)
        return random.choice(synonyms)
        
    #TODO: function to perform search against ConceptNet (requires API call using requests library)
    #TODO: function to generate random embedding element for missing word, refer to research paper